# I want to see which stars have surface rotation rates

In [1]:
import numpy as np
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import astropy.units as u
from astroquery.vizier import Vizier
from astropy.io import ascii
import warnings
warnings.filterwarnings('ignore')

sns.set_palette('colorblind')
sns.set_context('poster')

In [2]:
ati = pd.read_csv('../../data/atium.csv',index_col=0)
ati = ati[np.isfinite(ati.nus.values)]
print(f'We have {len(ati)} stars with results')

We have 95 stars with results


In [3]:
lit = pd.DataFrame()
lit['KIC'] = ati.KIC.copy()
lit['Flag'] = np.nan

### Read in the McQuillan+ sample

In [4]:
cat = Vizier(catalog='J/ApJS/211/24/table1', 
             columns=['KIC', 'Prot', 'e_Prot'])
cat.ROW_LIMIT = -1
mcq = cat.get_catalogs(cat.catalog)[0].to_pandas()
mcq = mcq.rename(columns={'Prot':'McQP','e_Prot':'e_McQP'})

In [5]:
mcq.head(2)

,KIC,McQP,e_McQP
0,892376,1.532,0.007
1,1026146,14.891,0.120


#### Merge results in

In [6]:
lit = pd.merge(lit, mcq, how='left', on='KIC')

In [7]:
lit.loc[np.isfinite(lit.McQP), 'Flag'] = 1

In [8]:
print(f'We have {len(lit[np.isfinite(lit["McQP"])])} / {len(lit)} periods from McQuillan+15')

We have 10 / 95 periods from McQuillan+15


### Read in the Santos+19 sample

In [9]:
santos19 = ascii.read('../../data/santos19.txt').to_pandas()
santos19 = santos19[['KIC', 'Prot', 'E_Prot']]
santos19 = santos19.rename(columns={'Prot':'SP','E_Prot':'e_SP'})

In [10]:
santos19.head(2)

,KIC,SP,e_SP
0,892834,13.61,1.02
1,892882,21.96,1.70


#### Merge results in

In [11]:
lit = pd.merge(lit, santos19, how='left', on='KIC')
lit.loc[np.isfinite(lit.SP), 'Flag'] = 1

In [12]:
print(f'We have {len(lit[np.isfinite(lit["SP"])])} / {len(lit)} periods from Santos+19')

We have 0 / 95 periods from Santos+19


### Read in the Nielsen+13 sample

In [13]:
cat = Vizier(catalog='J/A+A/557/L10/table1', 
             columns=['KIC', 'Prot', 'MAD'])
cat.ROW_LIMIT = -1
n13 = cat.get_catalogs(cat.catalog)[0].to_pandas()
n13 = n13.rename(columns={'Prot':'NP','MAD':'e_NP'})

In [14]:
n13.head(2)

,KIC,NP,e_NP
0,891916,5.539,0.191
1,892834,13.663,0.098


#### Merge results in

In [15]:
lit = pd.merge(lit, n13, how='left', on='KIC')
lit.loc[np.isfinite(lit.NP), 'Flag'] = 1

In [16]:
print(f'We have {len(lit[np.isfinite(lit["NP"])])} / {len(lit)} periods')

We have 7 / 95 periods


### Read in the van Saders+16 sample

Don't forget to exclude 16 Cyg A & B, which had seismic rotation.

In [17]:
jvs = pd.read_csv('../../data/jvs_rot.csv', index_col=None)
jvs.dropna(inplace=True)
jvs.KIC = jvs.KIC.astype(int)
jvs = jvs[2:]
jvs.head(2)

,KIC,jvsP,e_jvsP
3,3427720,13.9,2.1
4,3656476,31.7,3.5


#### Merge results in


In [18]:
lit = pd.merge(lit, jvs, how='left', on='KIC')
lit.loc[np.isfinite(lit.jvsP), 'Flag'] = 1
print(f'We have {len(lit[np.isfinite(lit["jvsP"])])} / {len(lit)} periods from van Saders+16')

We have 18 / 95 periods from van Saders+16


### Read in the Garcia+14 sample
This should overlap fully with the van Saders sample.

In [19]:
cat = Vizier(catalog='	J/A+A/572/A34/table3', 
             columns=['KIC', 'Prot', 'e_Prot'])
cat.ROW_LIMIT = -1
g14 = cat.get_catalogs(cat.catalog)[0].to_pandas()
g14 = g14.rename(columns={'Prot':'GP','e_Prot':'e_GP'})
g14.head(2)

,KIC,GP,e_GP
0,1430163,4.16,0.92
1,1435467,6.68,0.89


#### Merge results in


In [20]:
lit = pd.merge(lit, g14, how='left', on='KIC')
print(f'We have {len(lit[np.isfinite(lit["GP"])])} / {len(lit)} periods from van Garcia+14')

We have 40 / 95 periods from van Garcia+14


### Flag stars with rotation rates

In [56]:
lit['Flag'] = np.nan
l = ['McQP', 'SP','NP','jvsP','GP']

for idx in lit.index:
    if any(np.isfinite(lit[l].loc[idx].values)):
        lit.loc[idx, 'Flag'] = 1
    else:
        lit.loc[idx, 'Flag'] = 0
lit['Flag'] = lit['Flag'].astype(int)

### Save out results

In [57]:
lit.to_csv('../../data/lit_spot_rots.csv')

In [58]:
lit

,KIC,Flag,McQP,e_McQP,SP,e_SP,NP,e_NP,jvsP,e_jvsP,GP,e_GP
0,1435467,1,NaN,NaN,NaN,NaN,6.955,0.392,NaN,NaN,6.680000,0.89
1,2837475,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.680000,0.36
2,3427720,1,NaN,NaN,NaN,NaN,NaN,NaN,13.9,2.1,NaN,NaN
3,3456181,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3632418,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.890000,1.38
5,3656476,1,NaN,NaN,NaN,NaN,NaN,NaN,31.7,3.5,31.670000,3.53
6,3735871,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.530000,1.24
7,4914923,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.490000,2.82
8,5184732,1,NaN,NaN,NaN,NaN,NaN,NaN,19.8,2.4,19.790001,2.43
9,5773345,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.570000,1.02
